In [1]:
import csv
import numpy as np
import pandas as pd
import pprint  
import collections
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
lm=linear_model.LinearRegression()

In [2]:
segment2_df = pd.DataFrame(pd.read_csv("segment2_processing_time.csv"))
pd.set_option('display.max_columns', 80)
print(segment2_df.shape)
print(len(segment2_df['CASE_ID'].unique()))

(29623, 58)
2133


E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
segment2_df.head(3)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,CreateTime,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,REFERRAL_ID,APPLICATION_ID,APPLICATION_LINE_ID,STU_ID,STU_CITIZEN_CD,STU_CITIZEN_NM,STU_BIRTH_DT,STU_ABOR_TSI_CD,STU_ABOR_TSI_NM,STU_BIRTH_CNTRY_CD,STU_BIRTH_CNTRY_NM,STU_CTZN_CNTRY_CD,STU_CTZN_CNTRY_NM,SUBMISSION_METHOD,ENTRY_TYPE,ASSESSMENT_DECISION_SCORE,ASSESSMENT_DECISION_NM,ASSESSMENT_OUTCOME_NM,LAST_COUNTRY_EDUCATION,PREV_INST_NM_MAIN,PREV_INST_NM_ALT1,PREV_INST_NM_ALT2,PREFERENCE_NO,SPK_CD,SPK_FULL_TITLE,ORG_UNIT_CD,AVAIL_ORG_UNIT_NM,SUPPORTING_ORG,STUDY_MODE_CD,RETURNING_DEFERRAL,CURR_AVAIL_YR,CURR_STUDY_PERIOD,CURR_SPRD_CD,CURR_SPRD_NM,ORIG_AVAIL_YR,ORIG_STUDY_PERIOD,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,ProcessingTime
0,646035|228799|364549,Defer,Defer,31/1/2018 10:59:31,2018-01-31 10:59:31,ATKINSONM,Faculty,1,Y,4. Post offer,Part 2 - Application Line ID Workflow,NaN,228799,364549,646035,8,Permanent Humanitarian Visa,28/2/1990 0:00:00,N,No,7106.0,Pakistan,7106.0,Pakistan,Online Application,Bachelor (equiv Aust 3yr deg),76.09,Qualified,Successful,Pakistan,Pakistan Institute of Engineering and Applied ...,Pakistan Institute of Engineering and Applied ...,Pakistan Institute of Engineering and Applied ...,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,Y,2019,Start Year Intake,STYI,Start Year Intake,2017,Mid Year Intake,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,AS,AS,Y,$DEF,N,0
1,646035|228799|364549,Release Offer,Release Offer,31/1/2018 10:59:22,2018-01-31 10:59:22,ATKINSONM,Faculty,2,N,4. Post offer,Part 2 - Application Line ID Workflow,NaN,228799,364549,646035,8,Permanent Humanitarian Visa,28/2/1990 0:00:00,N,No,7106.0,Pakistan,7106.0,Pakistan,Online Application,Bachelor (equiv Aust 3yr deg),76.09,Qualified,Successful,Pakistan,Pakistan Institute of Engineering and Applied ...,Pakistan Institute of Engineering and Applied ...,Pakistan Institute of Engineering and Applied ...,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,Y,2019,Start Year Intake,STYI,Start Year Intake,2017,Mid Year Intake,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,AS,AS,Y,$DEF,N,0 days 00:00:09
2,646035|228799|364549,Restart Offer Workflow,Restart Offer Workflow,31/1/2018 10:59:05,2018-01-31 10:59:05,646035,NaN,3,N,4. Post offer,Part 2 - Application Line ID Workflow,NaN,228799,364549,646035,8,Permanent Humanitarian Visa,28/2/1990 0:00:00,N,No,7106.0,Pakistan,7106.0,Pakistan,Online Application,Bachelor (equiv Aust 3yr deg),76.09,Qualified,Successful,Pakistan,Pakistan Institute of Engineering and Applied ...,Pakistan Institute of Engineering and Applied ...,Pakistan Institute of Engineering and Applied ...,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,Y,2019,Start Year Intake,STYI,Start Year Intake,2017,Mid Year Intake,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,AS,AS,Y,$DEF,N,0 days 00:00:17


In [4]:
segment2_info = dict()
for i in range(segment2_df.shape[0]):
    for ids in segment2_df['CASE_ID'].unique():
        if segment2_df['CASE_ID'].iloc[i] == ids:
            if ids not in segment2_info:
                segment2_info[ids] = [segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i]]
            else:
                segment2_info[ids] += [segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i]]                  

#### Workflow Segment 1

In [6]:
ws1_index = dict()
for i in range(segment2_df.shape[0]):
    for ids in segment2_df['CASE_ID'].unique():
        if ids == segment2_df['CASE_ID'].iloc[i]:
            if ids not in ws1_index:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Send for Assessment':
                    flag2 = i
                    ws1_index[ids] = [flag2]
            else:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Submit':
                    flag1 = i
                    ws1_index[ids] += [flag1]
                    
print(len(segment2_info))
print(len(ws1_index))

2133
1896


In [7]:
ws1_list = dict()
for k,v in ws1_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment2_df.loc[End_index:Start_index]
        ws1_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [9]:
ws1_info = pd.DataFrame(
    [[k,v] for k,l in ws1_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws1_info))

15136


In [10]:
ws1_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,646035|228799|364549,Send for Assessment
1,646035|228799|364549,AR - Satisfied - ELR
2,646035|228799|364549,AR - Satisfied - advanced standing
3,646035|228799|364549,AR - Satisfied - previous or current study
4,646035|228799|364549,Submit


In [11]:
ws1_dict = dict()
for k,v in ws1_list.items():
    for s in v.tolist():
        if k not in ws1_dict:
            ws1_dict[k] = [s]
        else:
            ws1_dict[k] += [s]

In [13]:
ws1_list_names=list()
for i in range(ws1_info.shape[0]-1, -1, -1):
    if ws1_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws1_list_names:
        ws1_list_names.append(ws1_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws1_list_names))
print(len(ws1_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

69
69


In [14]:
"""
A Dateframe:
    Column：all unique workflow names appeared in workflow segment1
    Row：all unique applicants in segment2
"""
segment2_ws1_feature = pd.DataFrame(columns=ws1_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment2_ws1_feature.shape)
ws1_feature_cols = segment2_ws1_feature.columns.values.tolist()
print(len(ws1_feature_cols))

(0, 69)
69


In [15]:
segment2_ws1_feature.insert(0,'CASE_ID',[k for k in segment2_info.keys()])

In [16]:
print(segment2_ws1_feature.shape)
segment2_ws1_feature.loc[:,1:70] = 0
segment2_ws1_feature = segment2_ws1_feature.set_index('CASE_ID')

(2133, 70)


In [17]:
segment2_ws1_feature.head(3)

,Send for Assessment,AR - Satisfied - ELR,AR - Satisfied - advanced standing,AR - Satisfied - previous or current study,Submit,OC - Created - Other,AR - Satisfied - entry points,Release Triage Hold,OC - Satisfied - Other,AR - Satisfied - work experience,Put Triage On-Hold,AR - Satisfied - further information,AR - Created - further information,AR - Created - work experience,AR - Satisfied - specialisation,AR - Satisfied - application details,AR - Not Satisfied - ELR,Back to Triage,Assessment Decision - Qualified,End My Assessment,AR - Satisfied - syllabus,Release Assessment Hold,AR - Created - previous or current study,Put Assessment On-Hold,Assessment Decision - No Decision,AR - Created - Other,AR - Satisfied - GAM,Assessment Outcome - Ineligible / Withdrawn,Assessment Decision - Not Qualified,Assessment Outcome - Not Successful,AR - Satisfied - Other,Acacdemic - Assessment Decision - Qualified,AR - Reviewed - specialisation,AR - Reviewed - entry points,AR - Reviewed - application details,OC - Reviewed - Other,AR - Reviewed - syllabus,AR - Requested - advanced standing,OC - Satisfied - further information,OC - Created - further information,AR - Satisfied - agent,AR - Reviewed - GAM,AR - Requested - further information,AR - Created - entry points,AR - Created - specialisation,"AR - Satisfied - financial (sponsorship, scholarship etc)","AR - Reviewed - financial (sponsorship, scholarship etc)",Acacdemic - Assessment Decision - No Decision,AR - Satisfied - guaranteed entry,AR - Created - advanced standing,AR - Satisfied - applicant demographics,AR - Created - applicant demographics,AR - Reviewed - advanced standing,AR - Not Satisfied - previous or current study,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - Not Assessed,Release Offer,Create Offer,Assessment Outcome - Successful,AR - Requested - syllabus,AR - Reviewed - previous or current study,AR - Requested - previous or current study,AR - Reviewed - ELR,AR - Requested - ELR,Assessment Decision - System Assessed,OC - Created - ELR,AR - Reviewed - Other,AR - Reviewed - further information,AR - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
646035|228799|364549,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
935028|237355|377891,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
927815|252562|402189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [84]:
for k in segment2_info.keys():
    for v in segment2_info[k]:
        if v in ws1_feature_cols:
            segment2_ws1_feature.loc[k,v] = 1
        else:
            continue

In [85]:
print(segment2_ws1_feature.shape)
print(segment2_ws1_feature['Send for Assessment'].unique())

(2133, 69)
[1 0]


In [86]:
segment2_ws1_feature.head(3)

,Send for Assessment,AR - Satisfied - ELR,AR - Satisfied - advanced standing,AR - Satisfied - previous or current study,Submit,OC - Created - Other,AR - Satisfied - entry points,Release Triage Hold,OC - Satisfied - Other,AR - Satisfied - work experience,Put Triage On-Hold,AR - Satisfied - further information,AR - Created - further information,AR - Created - work experience,AR - Satisfied - specialisation,AR - Satisfied - application details,AR - Not Satisfied - ELR,Back to Triage,Assessment Decision - Qualified,End My Assessment,AR - Satisfied - syllabus,Release Assessment Hold,AR - Created - previous or current study,Put Assessment On-Hold,Assessment Decision - No Decision,AR - Created - Other,AR - Satisfied - GAM,Assessment Outcome - Ineligible / Withdrawn,Assessment Decision - Not Qualified,Assessment Outcome - Not Successful,AR - Satisfied - Other,Acacdemic - Assessment Decision - Qualified,AR - Reviewed - specialisation,AR - Reviewed - entry points,AR - Reviewed - application details,OC - Reviewed - Other,AR - Reviewed - syllabus,AR - Requested - advanced standing,OC - Satisfied - further information,OC - Created - further information,AR - Satisfied - agent,AR - Reviewed - GAM,AR - Requested - further information,AR - Created - entry points,AR - Created - specialisation,"AR - Satisfied - financial (sponsorship, scholarship etc)","AR - Reviewed - financial (sponsorship, scholarship etc)",Acacdemic - Assessment Decision - No Decision,AR - Satisfied - guaranteed entry,AR - Created - advanced standing,AR - Satisfied - applicant demographics,AR - Created - applicant demographics,AR - Reviewed - advanced standing,AR - Not Satisfied - previous or current study,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - Not Assessed,Release Offer,Create Offer,Assessment Outcome - Successful,AR - Requested - syllabus,AR - Reviewed - previous or current study,AR - Requested - previous or current study,AR - Reviewed - ELR,AR - Requested - ELR,Assessment Decision - System Assessed,OC - Created - ELR,AR - Reviewed - Other,AR - Reviewed - further information,AR - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
646035|228799|364549,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0
935028|237355|377891,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
927815|252562|402189,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Workflow Segment2

In [30]:
ws2_index = dict()
for i in range(segment2_df.shape[0]):
    for ids in segment2_df['CASE_ID'].unique():
        if ids == segment2_df['CASE_ID'].iloc[i]:
            if ids not in ws2_index:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Assessment Outcome - Successful':
                    flag2 = i
                    ws2_index[ids] = [flag2]
                elif segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Assessment Outcome - Not Successful':
                    flag3 = i
                    ws2_index[ids] = [flag3]
            else:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Send for Assessment':
                    flag1 = i
                    ws2_index[ids] += [flag1]

In [31]:
print(len(ws2_index))

1486


In [32]:
ws2_list = dict()
for k,v in ws2_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment2_df.loc[End_index:Start_index]
        ws2_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [34]:
ws2_info = pd.DataFrame(
    [[k,v] for k,l in ws2_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws2_info))

6970


In [35]:
ws2_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,646035|228799|364549,Assessment Outcome - Successful
1,646035|228799|364549,Assessment Decision - Qualified
2,646035|228799|364549,Assessment Decision - Qualified
3,646035|228799|364549,Assessment Decision - Qualified
4,646035|228799|364549,Acacdemic - Assessment Decision - Qualified


In [36]:
ws2_dict = dict()
for k,v in ws2_list.items():
    for s in v.tolist():
        if k not in ws2_dict:
            ws2_dict[k] = [s]
        else:
            ws2_dict[k] += [s]

In [38]:
ws2_list_names=list()
for i in range(ws2_info.shape[0]-1, -1, -1):
    if ws2_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws2_list_names:
        ws2_list_names.append(ws2_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws2_list_names))
print(len(ws2_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

49
49


In [39]:
"""
A Dateframe:
    Column：all unique workflow names appeared in workflow segment2
    Row：all unique applicants in segment2
"""
segment2_ws2_feature = pd.DataFrame(columns=ws2_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment2_ws2_feature.shape)

ws2_feature_cols = segment2_ws2_feature.columns.values.tolist()
print(len(ws2_feature_cols))

(0, 49)
49


In [40]:
segment2_ws2_feature.insert(0,'CASE_ID',[k for k in segment2_info.keys()])

In [41]:
print(segment2_ws2_feature.shape)
segment2_ws2_feature.loc[:,1:50] = 0
segment2_ws2_feature = segment2_ws2_feature.set_index('CASE_ID')

(2133, 50)


In [42]:
segment2_ws2_feature.head(3)

,Assessment Outcome - Successful,Assessment Decision - Qualified,Acacdemic - Assessment Decision - Qualified,Send for Assessment,End My Assessment,Assessment Outcome - Not Successful,Assessment Decision - Not Qualified,Assessment Decision - No Decision,Release Assessment Hold,Put Assessment On-Hold,AR - Satisfied - entry points,Acacdemic - Assessment Decision - No Decision,OC - Created - Other,Acacdemic - Assessment Decision - Not Qualified,AR - Satisfied - previous or current study,Release Triage Hold,AR - Satisfied - further information,Put Triage On-Hold,Back to Triage,AR - Satisfied - advanced standing,AR - Created - previous or current study,AR - Created - further information,AR - Satisfied - syllabus,AR - Created - Other,Assessment Outcome - Higher Preference Successful,AR - Satisfied - ELR,AR - Not Satisfied - ELR,AR - Satisfied - Other,AR - Satisfied - specialisation,Create Offer,AR - Satisfied - application details,AR - Satisfied - GAM,AR - Created - specialisation,Acacdemic - Assessment Outcome - No Outcome Set,AR - Created - GAM,AR - Reviewed - advanced standing,OC - Satisfied - Other,Assessment Decision - System Assessed,Assessment Outcome - Ineligible / Withdrawn,AR - Reviewed - application details,Acacdemic - Assessment Decision - System Assessed,Assessment Decision - Not Assessed,Restart Offer Workflow,Cancel,OC - Not Satisfied - Other,Accept,Manage Acceptance,Release Offer,AR - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
646035|228799|364549,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
935028|237355|377891,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
927815|252562|402189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
for k in segment2_info.keys():
    for v in segment2_info[k]:
        if v in ws2_feature_cols:
            segment2_ws2_feature.loc[k,v] = 1
        else:
            continue

In [44]:
print(segment2_ws2_feature.shape)
print(segment2_ws2_feature['Assessment Outcome - Successful'].unique())
print(segment2_ws2_feature['Assessment Outcome - Not Successful'].unique())

(2133, 49)
[1 0]
[0 1]


#### Workflow Segment 3

In [48]:
ws3_index = dict()
for i in range(segment2_df.shape[0]):
    for ids in segment2_df['CASE_ID'].unique():
        if ids == segment2_df['CASE_ID'].iloc[i]:
            if ids not in ws3_index:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Release Offer':
                    flag2 = i
                    ws3_index[ids] = [flag2]
            else:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Assessment Outcome - Successful':
                    flag1 = i
                    ws3_index[ids] += [flag1]

print(len(segment2_info))
print(len(ws3_index))

2133
1130


In [50]:
ws3_list = dict()
for k,v in ws3_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment2_df.loc[End_index:Start_index]
        ws3_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [52]:
ws3_info = pd.DataFrame(
    [[k,v] for k,l in ws3_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws3_info))

4596


In [53]:
ws3_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,646035|228799|364549,Release Offer
1,646035|228799|364549,Restart Offer Workflow
2,646035|228799|364549,Allow Workflow Restart
3,646035|228799|364549,Auto Finalise
4,646035|228799|364549,Accept


In [54]:
ws3_dict = dict()
for k,v in ws3_list.items():
    for s in v.tolist():
        if k not in ws3_dict:
            ws3_dict[k] = [s]
        else:
            ws3_dict[k] += [s]

In [56]:
ws3_list_names=list()
for i in range(ws3_info.shape[0]-1, -1, -1):
    if ws3_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws3_list_names:
        ws3_list_names.append(ws3_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws3_list_names))
print(len(ws3_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

26
26


In [57]:
"""
A Dateframe:
    Column：all unique workflow names appeared in workflow segment3 
    Row：all unique applicants in segment2
"""
segment2_ws3_feature = pd.DataFrame(columns=ws3_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment2_ws3_feature.shape)

ws3_feature_cols = segment2_ws3_feature.columns.values.tolist()
print(len(ws3_feature_cols))

(0, 26)
26


In [58]:
segment2_ws3_feature.insert(0,'CASE_ID',[k for k in segment2_info.keys()])
print(segment2_ws3_feature.shape)

(2133, 27)


In [59]:
segment2_ws3_feature.loc[:,1:27] = 0
segment2_ws3_feature = segment2_ws3_feature.set_index('CASE_ID')
print(segment2_ws3_feature.shape)

(2133, 26)


In [60]:
segment2_ws3_feature.head(3)

,Release Offer,Restart Offer Workflow,Allow Workflow Restart,Auto Finalise,Accept,Create Offer,Assessment Outcome - Successful,Finalise,OC - Satisfied - Other,Manage Acceptance,Assessment Decision - No Decision,Ready to Finalise,Defer,Assessment Decision - Qualified,OC - Created - Other,Cancel,Acacdemic - Assessment Decision - Qualified,AR - Satisfied - ELR,Put Assessment On-Hold,AR - Reviewed - further information,AR - Created - further information,Release Assessment Hold,AR - Satisfied - further information,AR - Satisfied - previous or current study,Decline,AR - Satisfied - advanced standing
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
646035|228799|364549,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
935028|237355|377891,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
927815|252562|402189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
for k in segment2_info.keys():
    for v in segment2_info[k]:
        if v in ws3_feature_cols:
            segment2_ws3_feature.loc[k,v] = 1
        else:
            continue
            
print(segment2_ws3_feature.shape)
print(segment2_ws3_feature['Release Offer'].unique())

(2133, 26)
[1 0]


#### Workflow Segment 4

In [65]:
ws4_index = dict()
for i in range(segment2_df.shape[0]):
    for ids in segment2_df['CASE_ID'].unique():
        if ids == segment2_df['CASE_ID'].iloc[i]:           
            if ids not in ws4_index:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Accept':
                    flag2 = i
                    ws4_index[ids] = [flag2]
                elif segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Defer':
                    flag3 = i
                    ws4_index[ids] = [flag3]
            else:
                if segment2_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Release Offer':
                    flag1 = i
                    ws4_index[ids] += [flag1]

print(len(segment2_info))
print(len(ws4_index))

2133
900


In [67]:
ws4_list = dict()
for k,v in ws4_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment2_df.loc[End_index:Start_index]
        ws4_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [69]:
ws4_info = pd.DataFrame(
    [[k,v] for k,l in ws4_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws4_info))

3028


In [70]:
ws4_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,646035|228799|364549,Defer
1,646035|228799|364549,Release Offer
2,646035|228799|364549,Restart Offer Workflow
3,646035|228799|364549,Allow Workflow Restart
4,646035|228799|364549,Auto Finalise


In [71]:
ws4_dict = dict()
for k,v in ws4_list.items():
    for s in v.tolist():
        if k not in ws4_dict:
            ws4_dict[k] = [s]
        else:
            ws4_dict[k] += [s]

In [73]:
ws4_list_names=list()
for i in range(ws4_info.shape[0]-1, -1, -1):
    if ws4_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws4_list_names:
        ws4_list_names.append(ws1_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws4_list_names))
print(len(ws4_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

2878
37


In [74]:
"""
A Dateframe:
    Column：all unique workflow names appeared in workflow segment4 
    Row：all unique applicants in segment2
"""
segment2_ws4_feature = pd.DataFrame(columns=ws4_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment2_ws4_feature.shape)

ws4_feature_cols = segment2_ws4_feature.columns.values.tolist()
print(len(ws4_feature_cols))

(0, 37)
37


In [75]:
segment2_ws4_feature.insert(0,'CASE_ID',[k for k in segment2_info.keys()])

In [76]:
print(segment2_ws4_feature.shape)
segment2_ws4_feature.loc[:,1:38] = 0
segment2_ws4_feature = segment2_ws4_feature.set_index('CASE_ID')
print(segment2_ws4_feature.shape)

(2133, 38)
(2133, 37)


In [77]:
segment2_ws4_feature.head(3)

,Defer,Release Offer,Restart Offer Workflow,Allow Workflow Restart,Auto Finalise,Accept,Finalise,OC - Satisfied - Other,Manage Acceptance,Assessment Decision - No Decision,Ready to Finalise,Assessment Decision - Qualified,OC - Created - Other,Cancel,AR - Satisfied - syllabus,Acacdemic - Assessment Decision - Qualified,AR - Reviewed - further information,AR - Created - further information,AR - Satisfied - ELR,AR - Satisfied - advanced standing,AR - Satisfied - previous or current study,AR - Satisfied - further information,AR - Reviewed - ELR,OC - Requested - Other,AR - Reviewed - GAM,End My Assessment,Send for Assessment,AR - Created - previous or current study,Release Assessment Hold,Put Assessment On-Hold,AR - Satisfied - application details,AR - Satisfied - GAM,OC - Reviewed - Other,AR - Satisfied - Other,AR - Reviewed - advanced standing,AR - Not Satisfied - ELR,AR - Reviewed - syllabus
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
646035|228799|364549,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
935028|237355|377891,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
927815|252562|402189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
for k in segment2_info.keys():
    for v in segment2_info[k]:
        if v in ws4_feature_cols:
            segment2_ws4_feature.loc[k,v] = 1
        else:
            continue
            
print(segment2_ws4_feature.shape)
print(segment2_ws4_feature['Accept'].unique())
print(segment2_ws4_feature['Defer'].unique())

(2133, 37)
[1 0]
[1 0]
